# Лабораторная работа 1
### Выполнил: Рухлядев Алексей Павлович

Добро пожаловать в лабораторную работу!
Оформите ваше решение контеста Kaggle в соответствии с этим шаблоном. Ваша задача - показать проверяющим ход ваших рассуждений, поэтому советуем писать много комментариев к коду и приводить комментарии к логике на каждом этапе в текстовых ячейках.
Где необходимо, используйте графики для большей наглядности.

При отправке поменяйте название файла на ваши ФИО!

### EDA (исследовательский анализ данных)

В этом разделе вам необходимо провести анализ вашего датасета, интерпретировать признаки, выяснить их значимость и исследовать зависимости между ними.

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns


my_seed = 42
random.seed(my_seed)
np.random.seed(my_seed)

In [ ]:
# Для начала просто посмотрим на данные и их формат
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
train.info()
train.head()

In [ ]:
train.describe()

In [ ]:
for col in ['cut', 'color', 'clarity']:
    print(f"{col}: {train[col].unique()}")

Теперь пробежимся по самим признакам (кроме целевой переменной цены `price`):
1. `carat` - масса алмаза в каратах. Числовой признак
2. `cut` - уровень огранки. Категориальный признак, придется побить на кучу столбцов
3. `color` - цвет алмаза. Категориальный - бьем
4. `clarity`- мера чистоты (в смысле наличия включений) алмаза. Категориальный - бьем. Есть пропуски

Ссылка мне в помощь: https://www.bronnitsy.com/brilliants/anatomy/

5. `depth` - глубина. Числовой. Есть пропуски
6. `table`- площадка. Числовой.
7. `x`, `y`, `z` - размер по осям. Числовые. Есть пропуски. Также в `y` и `z` есть какие-то неприятные выбросы.
8. `theta1`, `theta2` - ???. Числовые (правда, почему-то они заданы дискретными значениями, возможно, стоит их все-таки рассматривать категориальными)

In [ ]:
# Посмотрим на распределение числовых признаков
numeric_features = ['carat', 'depth', 'table', 'x', 'y', 'z', 'theta1', 'theta2']

fig, axes = plt.subplots(4, 2, figsize=(18, 12))
axes = axes.ravel()

for i, feature in enumerate(numeric_features):
    sns.histplot(train[feature], kde=True, ax=axes[i])
    axes[i].set_title(f'Распределение {feature}')

plt.tight_layout()
plt.show()

In [ ]:
# Для y и z отдельно избавимся от выбросов
fig, axes = plt.subplots(1, 2, figsize=(18, 5))
axes = axes.ravel()

def plot(df, x, idx):
    sns.histplot(df[x], kde=True, ax=axes[idx])

def untrash(val, df) -> pd.DataFrame:
    q1 = df[val].quantile(0.25)
    q3 = df[val].quantile(0.75)
    q = q3 - q1
    lb = q1 - 1.5 * q
    ub = q3 + 1.5 * q
    return df[(df[val] >= lb) & (df[val] <= ub)]

plot(untrash('y', train), 'y', 0)
plot(untrash('z', train), 'z', 1)
plt.tight_layout()
plt.show()

In [ ]:
# Посмотрим, где еще есть выбросы
fig, axes = plt.subplots(4, 2, figsize=(18, 12))
axes = axes.ravel()

for i, feature in enumerate(numeric_features):
    sns.boxplot(y=train[feature], ax=axes[i])
    axes[i].set_title(f'Выбросы в {feature}')

for i in range(len(numeric_features), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [ ]:
# Матрица корреляций для числовых признаков
numeric_df = train[numeric_features + ['price']]

for feature in numeric_features:
    numeric_df = untrash(feature, numeric_df)

plt.figure(figsize=(12, 8))
corr_matrix = numeric_df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, square=True, fmt='.2f')
plt.title('Матрица корреляций числовых признаков (с почищенными выбросами)')
plt.show()
# Логичное наблюдение: наиболее сильно цена зависит от массы и размера (что, в конечном счете, одно и то же).
# Также видно, что оптимальный размер площадки зависит от в принципе размера алмаза, а глубина - непосредственно от высоты (но не других размеров) всего алмаза

In [ ]:
# Посмотрим, как эти зависимости выглядят на графиках
from pandas.plotting import scatter_matrix
# Очень смешной рикролл, выкинем thet'ы отсюда
numeric_df.drop(['theta1', 'theta2'], axis=1, inplace=True)
scatter_matrix(numeric_df, figsize=(25, 20))
plt.show()
# Зависимости x,y,z друг от друга очень похожи на линейные. Логично, ведь формы огранки алмазов вполне определенные

In [ ]:
plt.figure(figsize=(4, 4))
plt.scatter(train['theta1'], train['theta2'])
plt.tight_layout()
plt.show()

In [ ]:
# Теперь посмотрим на категориальные признаки
categorial_features = ['cut', 'color', 'clarity']

fig, axes = plt.subplots(1, 3, figsize=(20, 8))

for i, feature in enumerate(categorial_features):
    sns.countplot(data=train, x=feature, ax=axes[i])
    axes[i].set_title(f'Распределение {feature}')
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 8))

for i, feature in enumerate(categorial_features):
    sns.boxplot(data=train, y=feature, x='price', ax=axes[i])

plt.tight_layout()
plt.show()

### Preprocessing (подготовка данных)

 В этом разделе вам необходимо реализовать подготовку ваших данных, в том числе заполнение пропусков, фильтрацию выбросов, кодирование категориальных признаков и т.д. В этот же раздел включите любые операции над данными, которые сочтете нужными.

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
x = train.drop(['id', 'price', 'theta1', 'theta2'], axis=1)
y = train['price']

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from pandas.core.series import Series

numeric_features = ['carat', 'depth', 'table', 'x', 'y', 'z']
categorial_features = ['cut', 'color', 'clarity']

def untrash(arr: Series) -> pd.DataFrame:
    lb = arr.quantile(0.01)
    ub = arr.quantile(0.99)
    return np.clip(arr, lb, ub)

# Чистим выбросы
def handle_outliers(df: pd.DataFrame) -> pd.DataFrame:
    dfc = df.copy()
    for feature in numeric_features:
        dfc[feature] = untrash(dfc[feature])
    return dfc

# Заменяем пропущенные значения
def handle_missing(df: pd.DataFrame) -> pd.DataFrame:
    preprocessor = ColumnTransformer(
        transformers=[
            ('num_imputer', SimpleImputer(strategy='median'), numeric_features),
            ('cat_imputer', SimpleImputer(strategy='most_frequent'), categorial_features)
        ],
        remainder='passthrough'
    )
    df_imputed = preprocessor.fit_transform(df)
    df_imputed = pd.DataFrame(df_imputed, columns=numeric_features + categorial_features)
    print(df.info())
    return df_imputed

# Приводим категориальные переменные к человеческому виду (можно было через get_dummies, но оно давало худший результат)
def get_dummies(df: pd.DataFrame) -> pd.DataFrame:
    dfc = df.copy()
    cut_mapping = {
        'Fair': 1,
        'Good': 2,
        "'Very Good'": 3,
        'Premium': 4,
        'Ideal': 5
    }

    # Для clarity используем порядковое кодирование
    clarity_mapping = {
        'I1': 1,
        'SI2': 2,
        'SI1': 3,
        'VS2': 4,
        'VS1': 5,
        'VVS2': 6,
        'VVS1': 7,
        'IF': 8
    }

    # Для color используем порядковое кодирование
    color_mapping = {
        'D': 1,
        'E': 2,
        'F': 3,
        'G': 4,
        'H': 5,
        'I': 6,
        'J': 7,
    }

    # Применяем кодирование
    dfc['cut'] = dfc['cut'].map(cut_mapping) * dfc['carat']
    dfc['clarity'] = dfc['clarity'].map(clarity_mapping) * dfc['carat']
    dfc['color'] = dfc['color'].map(color_mapping) * dfc['carat']
    return dfc

    # dfc = pd.get_dummies(df, columns=categorial_features)
    # for feature in dfc.columns:
    #     for f in categorial_features:
    #         if f in feature:
    #             dfc[feature] *= dfc['carat']
    # return dfc

# Немного feature-engineering'а
def get_new_features(df: pd.DataFrame) -> pd.DataFrame:
    poly = PolynomialFeatures(degree=2, include_bias=False)
    poly.fit(df)
    cols = poly.get_feature_names_out(df.columns)
    dfc = pd.DataFrame(poly.transform(df), columns=cols, index=df.index)
    xyz_features = ['x', 'y', 'z']

    from itertools import combinations_with_replacement
    triple_combinations = combinations_with_replacement(xyz_features, 3)

    for combo in triple_combinations:
        sorted_combo = sorted(combo)
        feature_name = '*'.join(sorted_combo)
        dfc[feature_name] = dfc[sorted_combo[0]] * dfc[sorted_combo[1]] * dfc[sorted_combo[2]]

    return dfc

# Хотел добавить стандартизацию, но она ужасно убивала скор (почему-то, до сих пор не нашел объяснения)
def scale_features(df: pd.DataFrame) -> pd.DataFrame:
    dfc = df.copy()
    scaler = StandardScaler()
    cols = dfc.columns[dfc.dtypes != 'bool']
    scaler.fit(dfc[cols])
    dfc[cols] = scaler.transform(dfc[cols])
    return dfc

# Ну и сразу сделаем функцию, чтобы удобно применить и к трейну, и к тесту
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    dfc = df.copy()
    dfc = handle_outliers(dfc)
    dfc = handle_missing(dfc)
    dfc = get_dummies(dfc)
    dfc = get_new_features(dfc)
    return dfc

In [ ]:
x = preprocess(x)
x

In [ ]:
x.info()

### Model & training (Выбор модели и её обучение)

В этом разделе описываете модель и ставите эксперименты по обучению.

Если вы ставили много экспериментов, приведите их в хронологическом порядке чтобы мы увидели эволюцию ваших рассуждений.

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error

#### Эксперимент 1

In [ ]:
# Для начала посмотрим на лин рег без регуляризации

kf = KFold(n_splits=5, shuffle=True, random_state=my_seed)

models = {
    'Linear Regression': LinearRegression()
}

def evaluate_models(models, X, y, cv):
    results = {}

    for name, model in models.items():
        print(f"\n--- Оценка {name} ---")

        # Кросс-валидация с MAE
        mae_scores = -cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error')

        results[name] = {
            'MAE': mae_scores.mean(),
            'MAE_std': mae_scores.std(),
            'model': model
        }

        print(f"MAE: {mae_scores.mean():.2f} ± {mae_scores.std():.2f}")

    return results

base_results = evaluate_models(models, x, y, kf)

#### Эксперимент 2

In [ ]:
# Посмотрим, насколько ситуацию улучшает регуляризация

models_reg = {
    'Ridge (alpha=0.1)': Ridge(alpha=0.1, random_state=my_seed),
    'Ridge (alpha=1.0)': Ridge(alpha=1.0, random_state=my_seed),
    'Ridge (alpha=10.0)': Ridge(alpha=10.0, random_state=my_seed),
    'Lasso (alpha=0.1)': Lasso(alpha=0.1, random_state=my_seed),
    'Lasso (alpha=1.0)': Lasso(alpha=1.0, random_state=my_seed),
}

reg_results = evaluate_models(models_reg, x, y, kf)

#### Эксперимент 3

In [ ]:
# Теперь подберем гиперпараметры для каждой из регуляризаций
param_grid = {
    'ridge': {'alpha': [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 20, 50]},
    'lasso': {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1]}
}

# Подбор для Ridge
print("--- Подбор гиперпараметров для Ridge ---")
ridge = Ridge(random_state=my_seed)
ridge_grid = GridSearchCV(ridge, param_grid['ridge'], cv=kf, scoring='neg_mean_absolute_error', n_jobs=-1)
ridge_grid.fit(x, y)

print(f"Лучшие параметры Ridge: {ridge_grid.best_params_}")
print(f"Лучший MAE: {-ridge_grid.best_score_:.2f}")

# Подбор для Lasso
print("--- Подбор гиперпараметров для Lasso ---")
lasso = Lasso(random_state=my_seed, max_iter=10000)
lasso_grid = GridSearchCV(lasso, param_grid['lasso'], cv=kf, scoring='neg_mean_absolute_error', n_jobs=-1)
lasso_grid.fit(x, y)

print(f"Лучшие параметры Lasso: {lasso_grid.best_params_}")
print(f"Лучший MAE: {-lasso_grid.best_score_:.2f}")

models_tuned = {
    'Linear Regression': LinearRegression(),
    f'Ridge (alpha={ridge_grid.best_params_["alpha"]})': Ridge(alpha=ridge_grid.best_params_['alpha'], random_state=my_seed),
    f'Lasso (alpha={lasso_grid.best_params_["alpha"]})': Lasso(alpha=lasso_grid.best_params_['alpha'], random_state=my_seed, max_iter=10000),
}

tuned_results = evaluate_models(models_tuned, x, y, kf)

#### Выбор лучшей модели

In [ ]:
X_test = preprocess(test.drop(['id', 'theta1', 'theta2'], axis=1))

comparison_df = pd.DataFrame({
    model: {
        'MAE': results['MAE'],
        'MAE_std': results['MAE_std'],
    }
    for model, results in tuned_results.items()
}).T.sort_values('MAE')

best_model_name = comparison_df.index[0]
best_model = tuned_results[best_model_name]['model']
best_mae = comparison_df.loc[best_model_name, 'MAE']

print(f"--- Финальное обучение {best_model_name} на всех данных ---")
final_model = best_model

X_test_final = X_test
final_model.fit(x, y)

In [ ]:
y_pred_test = final_model.predict(X_test_final)

print(f"Диапазон предсказаний: {y_pred_test.min():.2f} - {y_pred_test.max():.2f}")
print(f"Среднее предсказание: {y_pred_test.mean():.2f}")

submission = pd.DataFrame({
    'id': test['id'],
    'price': y_pred_test
})

submission_file = 'submission.csv'
submission.to_csv(submission_file, index=False)

### Evaluation (оценка качества модели)

В этом разделе проводите оценку качества вашей итоговой модели.

In [ ]:
pred = final_model.predict(x)

mae = mean_absolute_error(y, pred)
mae

In [ ]:
residuals = y - pred

fig, axes = plt.subplots(1, 2, figsize=(18, 5))

axes[0].scatter(pred, residuals)
axes[0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0].set_xlabel('Предсказание')
axes[0].set_ylabel('Остаток')
axes[0].set_title('График остатков')
axes[0].grid(alpha=0.3)

axes[1].hist(residuals, bins=50)
axes[1].set_title('Распределение остатков')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Среднее остатков: {residuals.mean():.6f}")
print(f"Среднее квадратичное отклонение остатков: {residuals.std():.4f}")

### Conclusion (Выводы)

В этом разделе описываете полученные результаты и проводите анализ выполненной работы.
Что получилось / не получилось и почему?

В ходе работы была достигнута MAE около 313 на тренировочных данных, что дало score ~455 на Kaggle. Для первого опыта участия в соревновании это можно считать успешным результатом.

Эффективность подходов:

- Наилучшие результаты показала линейная регрессия без регуляризации
- Feature-engineering с полиномиальными признаками и взаимодействиями значительно улучшил качество модели
- Порядковое кодирование категориальных переменных с умножением на вес карата оказалось эффективнее one-hot encoding
- По непонятным причинам стандартизация переменных делала модель очень плохой

Интересно было изучить смысл каждого столбца (никогда не интересовался алмазами с такой стороны). Особенно позабавила пасхалка в ненужных переменных `theta1` и `theta2`.